In [1]:
import pandas as pd
import os
import json
from key_word_extraction import *

In [2]:
dir_path = os.path.dirname(os.path.realpath("__file__"))
json_result = {}

In [5]:
import os
file_names = []
for file in os.listdir(dir_path):
    if file.endswith(".json") and "job_skill_distribution" not in file:
        file_names.append(os.path.join(dir_path, file))
        
file_names

['/home/ec2-user/web_scrape_data/data_6_24_boston.json',
 '/home/ec2-user/web_scrape_data/data_6_24_chicago.json',
 '/home/ec2-user/web_scrape_data/data_6_24_dc.json',
 '/home/ec2-user/web_scrape_data/data_6_24_ny.json',
 '/home/ec2-user/web_scrape_data/data_6_24_seattle.json',
 '/home/ec2-user/web_scrape_data/data_6_24_sf.json']

In [6]:
total_series = []
for file in file_names:

    city = file.split("_")[-1].split(".")[0]
    df = pd.read_json(file, orient="records")
    df = df.dropna(subset=["details"])
    result = extract_key_word_count(df).skill.value_counts()
    total_series.append(result)
    json_result[city] = result.to_dict()
    
total_result = total_series[0]
total_series.pop()
for series in total_series:
    total_result = total_result.add(series, fill_value=0)
json_result["total"] = total_result.to_dict()   

In [9]:
with open(dir_path + "/" + "job_skill_distribution.json", "w") as file:
    file.write(json.dumps(json_result, indent=4, sort_keys=True))